# Le type slice

---

*Peut être que certains d'entre vous trouvent étrange de rencontrer les slices aussi tard dans le programme. Elles sont en général enseignées bien en amont de beaucoup de parties que nous avons déjà vu, mais je trouve pour ma part que la compréhension des pointeurs et des références aide à mieux comprendre ce qu'est une slice.*

## Définition

> Une slice en Rust est **une référence sur un ensemble d'element**, on les considère comme des `fat pointers` ou pointeurs gras en français puisqu'elles possèdent dans leur définition, en plus d'une adresse mémoire, une taille.

- La syntaxe du type slice est composée du symbole de référencement `&` suivi du type entre crochets `[]` soit `&[type]` :

In [5]:
{
    let i: usize = 42;
    let reference_usize = &i;
    dbg!(std::mem::size_of::<&usize>()); // Une reference basique est une adresse memoire ...
                                         // ... et tient donc sur 8 octets sur un systeme 64 bits.
    
    let arr = [1, 2, 3, 4];
    let slice: &[u8] = &arr;
    dbg!(std::mem::size_of::<&[u8]>());  // En plus d'une adresse memoire, une slice possede ...
                                         // ... une indication sur sa taille qui tient aussi sur 8 octets.
                                         // La slice fait donc 16 octets sur un systeme 64 bits.
}

[src/lib.rs:25] std::mem::size_of::<&usize>() = 8
[src/lib.rs:30] std::mem::size_of::<&[u8]>() = 16


()

> Les slices vous permettent de référencer une séquence contiguë d'éléments dans une collection plutôt que la collection entière.

## Utilisation

### Les sous-slices

> La syntaxe à utiliser pour la création d'une slice repose sur l'utilisation du symbole de référencement `&` sur la variable dont on veut une slice. On peut y adjoindre des crochets apres le nom de la variable `[...SomeRange...]` afin de ne récupérer qu'une partie des éléments.

- Il est ainsi possible de ne "retenir" qu'une partie des éléments d'un tableau :

In [6]:
{
    let arr = [1, 2, 3, 4];
    let slice1: &[u8] = &arr; // Cette slice reference TOUT le tableau.
    dbg!(slice1);
    
    let slice2: &[u8] = &arr[1..=3]; // Seuls les 3 derniers elements du tableau sont references ici. 
    dbg!(slice2);
}

[src/lib.rs:25] slice1 = [
    1,
    3,
    2,
    4,
]
[src/lib.rs:28] slice2 = [
    2,
    3,
    4,
]


()

> Les possibilités syntaxiques sont :
```
- rien            -> Toute la collection est référencée.
- [..y], [..=y]   -> Référencement depuis le début jusqu'à l'index y, exclu ou inclu.
- [x..]           -> Référencement à partir de l'index x jusqu'à la fin.
- [x..y], [x..=y] -> Reférencement entre les index x et y, y est exclu ou inclu.
```

- Le code paniquera si la slice référence un index invalide :

In [15]:
{
    let arr = [0xff; 256];
    let slice = &arr[..512];
}

thread '<unnamed>' panicked at 'range end index 512 out of range for slice of length 256', src/lib.rs:24:18
stack backtrace:
   0: rust_begin_unwind
   1: core::panicking::panic_fmt
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/core/src/panicking.rs:64:14
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/std/src/panicking.rs:575:5
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/core/src/slice/index.rs:77:5
   2: core::slice::index::slice_end_index_len_fail_rt
   3: core::slice::index::slice_end_index_len_fail
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/core/src/slice/index.rs:69:9
   4: <unknown>
   5: evcxr::runtime::Runtime::run_loop
   6: evcxr::runtime::runtime_hook
   7: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


- Une slice peut référencer pleinement ou en partie une autre slice :

In [20]:
{
    let arr = [1, 2, 3, 4];
    println!("array len = {}", arr.len());
    let slice = &arr[1..];
    println!("slice len = {}", slice.len());
    let sub_slice = &slice[..2];
    println!("sub_slice len = {}", sub_slice.len());
    dbg!(sub_slice);
}

array len = 4
slice len = 3
sub_slice len = 2


[src/lib.rs:29] sub_slice = [
    2,
    3,
]


()

### L'emprunt et le non-transfert de possession

- Enfin comme les slices sont des références, il n'y aura pas de transfert de possession (ownership) des types réfèrés aux fonctions, méthodes ou closures. Voici un exemple qui utilise une allocation dynamique d'un tableau via la macro vec!, *je choisis cet exemple parce que les Vector ne sont pas Copy* :

In [13]:
{
    fn read_my_slice(slice: &[f64]) {
        dbg!(slice);
    }
    let arr_vec: Vec<f64> = vec!(1.2, 1.4, 1.6, 1.8); // Allocation dynamique d'une collection qui n'est pas Copy
    read_my_slice(&arr_vec);
    read_my_slice(&arr_vec[..1]);
    read_my_slice(&arr_vec[2..]);
    
    // j'ai toujours l'ownership de mon vecteur ici ...

    fn take_ownership(arr_vec: Vec<f64>) {} // Cette fonction prend l'ownership de ma collection
    take_ownership(arr_vec);

    // dbg!(arr_vec); // Cette ligne ne compile pas, puisque je viens de perdre l'ownership...
}

[src/lib.rs:24] slice = [
    1.2,
    1.6,
    1.4,
    1.8,
]
[src/lib.rs:24] slice = [
    1.2,
]
[src/lib.rs:24] slice = [
    1.6,
    1.8,
]


()

## Itération

- Il est possible de parcourir les éléments d'une slice dans une boucle for :

In [16]:
{
    let arr = [1, 2, 3, 4];
    let slice = &arr[1..];
    for elem in slice {
        dbg!(elem);
    }
}

[src/lib.rs:26] elem = 2
[src/lib.rs:26] elem = 3
[src/lib.rs:26] elem = 4


()

In [17]:
{
    let arr = [1, 2, 3, 4];
    let slice = &arr[1..];
    let sub_slice = &slice[1..];
    for elem in sub_slice {
        dbg!(elem);
    }
}

[src/lib.rs:27] elem = 3
[src/lib.rs:27] elem = 4


()

## Les méthodes

> Il y a près d'une centaine de méthodes pour les slices, les plus utilisées son len(), iter() et to_vec().

In [33]:
{
    let arr = [1, 2, 3, 4];
    let slice = &arr;
    dbg!(slice.len());
    let augmented_slice_iterator = slice.iter().map(|elem|
        elem * 2
    );
    dbg!(&augmented_slice_iterator);
    let v: Vec<usize> = augmented_slice_iterator.collect();
    println!("Vector form augmented slice");
    dbg!(v);
    println!("Vector from original slice");
    dbg!(slice.to_vec());
}

[src/lib.rs:28] slice.len() = 4
[src/lib.rs:32] &augmented_slice_iterator = Map {
    iter: Iter(
        [
            1,
            2,
            3,
            4,
        ],
    ),
}
[src/lib.rs:35] v = [
    2,
    4,
    6,
    8,
]
[src/lib.rs:37] slice.to_vec() = [
    1,
    2,
    3,
    4,
]


Vector form augmented slice
Vector from original slice


()

## Les accès mutables

- En utilisant `&mut` plutôt que `&` lors de sa création, un slice peut accéder en écriture à la variable référencée, les règles d'exclusivité des références mutables s'appliquent de la même façon pour les slices, pas d'accès autrement que par le slice mutable :

In [37]:
{
    let mut arr = [0, 1, 2, 3];
    let mutable_slice = &mut arr[..=1]; // La slice contient les deux premiers elements
    mutable_slice[0] = 42;
    // dbg!(arr); // La slice mutable vit encore, donc on ne peut pas acceder a l'array ainsi !!!
    mutable_slice[1] = 42;
    dbg!(arr);
}

[src/lib.rs:31] arr = [
    42,
    42,
    2,
    3,
]


()